In [294]:
import re
from natasha import (
    Segmenter,
    
    NewsMorphTagger,
    
    Doc
)
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from sklearn.metrics import accuracy_score
from nltk.util import ngrams 

In [168]:
with open('text_0.txt', 'r', encoding='utf-8') as t:
    text = t.read()

In [169]:
text = text.replace('\n', ' ')
text = text.replace('\u205f', ' ')
text_result = re.sub(r'[A-Za-z]', '', text)
text_ = re.sub(r'["«»]', '', text_result)
text_full = re.sub(r"\s[-—](\s)?", ' ', text_)

Данный текст подходит, так как есть словоформы с неизвестным корнем, но образованные с помощью продуктивных аффиксов (спотов, чиллит), есть словоформы с дефисом, в некоторых случаях его нельзя отделять (ей-богу, туда-сюда, перекати-поле), слова с неизвестными корнями, не содержащие продуктивных аффиксов (трэп, расслабон, грев), нестандартные формы (скрежет - может определится как глагол по окончанию или как существительное).

In [170]:
text_full

'И как я очутился, сдуру, где вечный сюр посередине жизни в сумеречном лесу? Я несу и так едва крест, я те не Иса. Где дорога? Ей-богу, дай упорюсь. Дай кредитку, не горячись и не кипишуй. Кто торчит? Меня, жить не учи, мы по чуть-чуть.   шик или китч, всё чересчур, судя по их глазам, все повеселели весьма. Челюсть туда-сюда, заледенела десна. Честно, я сам не знаю, ребзя, кем же я стал. Я всего лишь писатель, моё дело писать. Будапешт пропитан красным Данхиллом из . Ну как там вы, любимые мосты? Как там ты, брат-родник? Поднимаю голову вверх, чтобы увидеть на миг твой лик и солнца блик. Подмигнул на удачу, ты со мной навечно, брачо. Вокруг много спотов, но дом может быть лишь один. Мой дом где живут мои люди, но мои люди обжили весь мир. Кто-то живет в Торонто, кто-то чиллит в . Для кого-то дом это Бостон, мой отряд из Уфы. В моем загране так много штампов. Я не был в Риме, я не был в Тампе. Я был на небе, я был в капкане. Мне нужно место между двух палем. Перекати-поле, я как перекат

Тегсет был выбран на основе udpipe, как наиболеее универсальный для остальных трех парсеров (именно для этого тегсета не потребуется много исправлений). DET объединен с PRON, AUX c VERB и все союзы объединены в CONJ (так как некоторые парсеры не различают сочинительные и подчинительные союзы). 

In [75]:
with open('words.csv', 'r', encoding='utf-8') as t:
    words = t.read()
words = words.split('\n')
wordlist = []
for word in words:
    word_split = word.split(',')
    wordlist.append([word_split[0], word_split[1]])

# Mystem

In [171]:
m = Mystem()
ana = m.analyze(text_full)
word_pos_mystem = []
for word in ana:
    if 'analysis' in word:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        if pos == 'V':
            pos = 'VERB'
        if pos == 'S':
            pos = 'NOUN'
        if pos == 'A':
            pos = 'ADJ'
        if pos == 'SPRO' or pos == 'APRO':
            pos = 'PRON'
        if pos == 'PR':
            pos = 'ADP'
        if pos == 'ADVPRO':
            pos = 'CONJ'
        word_pos_mystem.append([word['text'], pos])

In [172]:
word_pos_mystem_clear = [] 
for word_pos in word_pos_mystem:
    if not (word_pos == ['понт', 'NOUN'] or word_pos == ['родник', 'NOUN']):
        word_pos_mystem_clear.append(word_pos)
# Mystem разделил две словоформы существительных, содержащих дефис, 
# поэтому было принято решение удалить появившиеся лишние токены, чтобы посчитать accuracy.
# Разделение на токены не повлияло на POS tagging, так как при разделении mystem указал 
# для двух частей словоформы одну и ту же часть речи

In [173]:
gold = []
for word_pos in wordlist:
    gold.append(word_pos[1])
results_mystem = []
for word_pos in word_pos_mystem_clear:
    results_mystem.append(word_pos[1])
accuracy_mystem = accuracy_score(gold, results_mystem)

In [174]:
accuracy_mystem

0.893491124260355

# Pymorphy2

In [175]:
morph = MorphAnalyzer()
text = text_full.replace('.', '')
text = text.replace('?', '')
text = text.replace(',', '')
text = text.lower()
text_splitted = text.split()
word_pos_pymorphy = []
for word in text_splitted:
    ana = morph.parse(word)
    first = ana[0]
    pos = first.tag.POS
    if pos == 'ADJF' or pos == 'ADJS':
        pos = 'ADJ'
    if pos == 'INFN' or pos == 'GRND':
        pos = 'VERB'
    if pos == 'NUMR':
        pos = 'NUM'
    if pos == 'ADVB':
        pos = 'ADV'
    if pos == 'NPRO':
        pos = 'PRON'
    if pos == 'PRCL':
        pos = 'PART'
    if pos == None:
        pos = 'NONE'
    word_pos_pymorphy.append([word, pos])

In [176]:
results_pymorphy = []
for word_pos in word_pos_pymorphy:
    results_pymorphy.append(word_pos[1])
accuracy_pymorphy = accuracy_score(gold, results_pymorphy)

In [177]:
accuracy_pymorphy

0.772189349112426

# Natasha

In [180]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc = Doc(text_full)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
word_pos_natasha = []
for token in doc.tokens:
    pos = token.pos
    if pos == 'SCONJ' or pos == 'CCONJ':
        pos = 'CONJ'
    if pos == 'AUX':
        pos = 'VERB'
    if pos == 'DET':
        pos = 'PRON'
    if pos != 'PUNCT':
        word_pos_natasha.append([token.text, pos])

In [181]:
results_natasha = []
for word_pos in word_pos_natasha:
    results_natasha.append(word_pos[1])
accuracy_natasha = accuracy_score(gold, results_natasha)

In [182]:
accuracy_natasha

0.8698224852071006

# Chunker for mystem

In [239]:
def chunker(word_pos_mystem, pos_1, pos_2):
    if pos_1 != 'не':
        token_1 = ''
        bigramm = []
        for i in range (0, len(word_pos_mystem)):
            if token_1 == '':
                if word_pos_mystem[i][1] == pos_1:
                    token_1 = word_pos_mystem[i][0]
            else:
                if word_pos_mystem[i][1] == pos_2 and word_pos_mystem[i][0][0].islower():
                    bigramm.append([token_1, word_pos_mystem[i][0]])
                    token_1 = ''
                elif word_pos_mystem[i][1] != pos_1:
                    token_1 = ''  
                else:
                    token_1 = word_pos_mystem[i][0]
                
    if pos_1 == 'не':
        bigramm = []
        for i in range (0, len(word_pos_mystem)):
            if word_pos_mystem[i][0] == part and (word_pos_mystem[i+1][1] == 'VERB' or word_pos_mystem[i+1][1] == 'NOUN'):
                bigramm.append([part, word_pos_mystem[i+1][0]])
            if word_pos_mystem[i][0] == part and word_pos_mystem[i+1][1] == 'ADJ' and word_pos_mystem[i+2][1] == 'NOUN':
                bigramm.append([part, word_pos_mystem[i+2][0]])
    
    return bigramm

In [246]:
adj_noun = chunker(word_pos_mystem, 'ADJ', 'NOUN')
neg = chunker(word_pos_mystem, 'не', None)
v_a = chunker(word_pos_mystem, 'VERB', 'ADV')
a_v = chunker(word_pos_mystem, 'ADV', 'VERB')
adv_verb = v_a + a_v

# Первое дз

In [247]:
# coding=UTF-8
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import requests
import re
import time
from tqdm import tqdm
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import accuracy_score

In [248]:
def writing_in_dict(soup):
    d_com = {}
    text_full = soup.find_all("div", attrs={"class": "responses__item__message markup-inside-small markup-inside-small--bullet"})
    regex = r"<.+>"
    comments_clean = []
    for each in text_full:
        result = re.sub(regex, '', str(each), 0, re.MULTILINE)
        result = result.replace('\n', '')
        result = result.replace('\t', '')
        result = result.replace('\xa0', '')
        comments_clean.append(result)
        time.sleep(random.uniform(3.2,5.4))
    return comments_clean

In [249]:
# x - оценка, y - страница 
database = {}
votes = [1,5]
for x in votes:
    for y in tqdm(range(1,4)):
        url = 'https://www.banki.ru/services/responses/bank/sberbank/?rate='+ str(x) +'&page='+ str(y) + '&isMobile=0'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        d = writing_in_dict(soup)
        a = str(x)+str(y)
        database[a] = d 

100%|██████████| 3/3 [05:23<00:00, 107.77s/it]


In [250]:
database['11'].extend(database['12'])
database['51'].extend(database['52'])
database['13'].extend(database['53'])

In [251]:
control_dict = {}
pos_list = []
neg_list = []
for each in database['13']:
    if each != '':
        if each in database['53']:
            pos_list.append(each)
        else:
            neg_list.append(each)
control_dict['pos'] = pos_list
control_dict['neg'] = neg_list

In [252]:
neg_coments = database['11']
pos_coments = database['51']
control_group = database['13']

In [253]:
neg_comments_30 = random.sample(neg_coments, 30)
pos_comments_30 = random.sample(pos_coments, 30)
control = random.sample(control_group, 10)

In [254]:
str_pos = ''.join(pos_comments_30)
str_neg = ''.join(neg_comments_30)
str_control = ''.join(control)

In [255]:
words_pos = [w.lower() for w in word_tokenize(str_pos) if w.isalpha()]
words_neg = [w.lower() for w in word_tokenize(str_neg) if w.isalpha()]
sw = stopwords.words('russian')
filtered_pos = [w for w in words_pos if w not in sw]
filtered_neg = [w for w in words_neg if w not in sw]

morph = MorphAnalyzer()
lemmas_pos = []
for word in filtered_pos:
    lemmas_pos.append(morph.parse(word)[0].normal_form)
lemmas_neg = []
for word in filtered_neg:
    lemmas_neg.append(morph.parse(word)[0].normal_form)

In [256]:
cnt_pos = Counter()
for word in lemmas_pos:
    cnt_pos[word] += 1
cnt_neg = Counter()
for word in lemmas_neg:
    cnt_neg[word] += 1

In [257]:
d_pos = {}
d_neg = {}
for key in dict(cnt_pos):
    if key not in dict(cnt_neg).keys() and cnt_pos[key]>1:
        d_pos[key] = cnt_pos[key]
for key in dict(cnt_neg):
    if key not in dict(cnt_pos).keys() and cnt_neg[key]>1:
        d_neg[key] = cnt_neg[key]

Были выделены следующие шаблоны: выразить + NOUN (на основе множественных примеров "выразить благодарность", "выразить восхищение") для положительного словаря;  VERB + что для негативного словаря, так как негативные отзывы характеризуются длинными описаниями ситуации (сказала, что; оказалось, что и т.д.). Универсальными стали шаблоны VERB + NOUN и ADJ + NOUN (соответственно, глаголы с прямыми дополнениями и прилагательные с существительными). То есть, получается действие + на что направлено действие (в основном) и описание + существительное. 

In [265]:
def chunker_neg(word_pos_mystem, pos_1, pos_2):
    if pos_1 == 'VERB' and pos_2 == 'что':
        bigramm = []
        for sent in word_pos_mystem:
            for i in range (0, len(sent)):
                try:
                    if sent[i][1] == pos_1 and sent[i+1][0] == pos_2:
                        bigramm.append([sent[i][0], sent[i+1][0]])
                except:
                    pass
    if (pos_1 == 'VERB' and pos_2 == 'NOUN') or (pos_1 == 'ADJ' and pos_2 == 'NOUN'):
        bigramm = []
        for sent in word_pos_mystem:
            for i in range (0, len(sent)):
                try:
                    if sent[i][1] == pos_1 and sent[i+1][1] == pos_2:
                        bigramm.append([sent[i][0], sent[i+1][0]])
                except:
                    pass
    return bigramm

In [266]:
def chunker_pos(word_pos_mystem, pos_1, pos_2):
    if pos_1 == 'выразить' and pos_2 == 'NOUN':
        bigramm = []
        for sent in word_pos_mystem:
            for i in range (0, len(sent)):
                try:
                    if sent[i][0] == pos_1 and sent[i+1][1] == 'ADJ' and sent[i+2][1] == pos_2:
                        bigramm.append([sent[i][0], sent[i+2][0]])
                except:
                    pass
    if (pos_1 == 'VERB' and pos_2 == 'NOUN') or (pos_1 == 'ADJ' and pos_2 == 'NOUN'):
        bigramm = []
        for sent in word_pos_mystem:
            for i in range (0, len(sent)):
                try:
                    if sent[i][1] == pos_1 and sent[i+1][1] == pos_2:
                        bigramm.append([sent[i][0], sent[i+1][0]])
                except:
                    pass
    return bigramm

In [267]:
sent = []
for comment in pos_coments:
    ana = m.analyze(comment)
    word_pos_mystem = []
    for word in ana:
        if 'analysis' in word:
            if word['analysis'] != []:
                gr = word['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                if pos == 'V':
                    pos = 'VERB'
                if pos == 'S':
                    pos = 'NOUN'
                if pos == 'A':
                    pos = 'ADJ'
                if pos == 'SPRO' or pos == 'APRO':
                    pos = 'PRON'
                if pos == 'PR':
                    pos = 'ADP'
                if pos == 'ADVPRO':
                    pos = 'CONJ'
            word_pos_mystem.append([word['text'], pos])
    sent.append(word_pos_mystem)
v_noun_pos = chunker_pos(sent, 'выразить', 'NOUN')
verb_noun = chunker_pos(sent, 'VERB', 'NOUN')
adj_noun = chunker_pos(sent, 'ADJ', 'NOUN')
bigram_pos = v_noun_pos + verb_noun + adj_noun

In [273]:
bigram_pos_clear = []
for bigram in bigram_pos:
    if bigram not in bigram_pos_clear:
        bigram_pos_clear.append(bigram)

In [282]:
sent_neg = []
for comment in neg_coments:
    ana = m.analyze(comment)
    word_neg_mystem = []
    for word in ana:
        if 'analysis' in word:
            if word['analysis'] != []:
                gr = word['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                if pos == 'V':
                    pos = 'VERB'
                if pos == 'S':
                    pos = 'NOUN'
                if pos == 'A':
                    pos = 'ADJ'
                if pos == 'SPRO' or pos == 'APRO':
                    pos = 'PRON'
                if pos == 'PR':
                    pos = 'ADP'
                if pos == 'ADVPRO':
                    pos = 'CONJ'
            word_neg_mystem.append([word['text'], pos])
    sent_neg.append(word_neg_mystem)
v_noun_neg = chunker_neg(sent_neg, 'VERB', 'что')
verb_noun = chunker_neg(sent_neg, 'VERB', 'NOUN')
adj_noun = chunker_neg(sent_neg, 'ADJ', 'NOUN')
bigram_neg = v_noun_neg + verb_noun + adj_noun

In [284]:
bigram_neg_clear = []
for bigram in bigram_neg:
    if bigram not in bigram_neg_clear:
        bigram_neg_clear.append(bigram)

In [285]:
freq_lists = {}
freq_lists['pos'] = d_pos
freq_lists['neg'] = d_neg

In [290]:
freq_lists_bg = {}
freq_lists_bg['pos'] = bigram_pos_clear
freq_lists_bg['neg'] = bigram_neg_clear

In [296]:
def detect_comment(freq_lists, text, freq_lists_bg):
    counts = Counter()
    for emotion, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
        filtered = [w for w in words if w not in sw]
        lemmas = []
        for word in filtered:
            lemmas.append(morph.parse(word)[0].normal_form)
        for word in lemmas:
            counts[emotion] += int(freq_list[word] > 0)
            
    for emotion, freq_list in freq_lists.items():
        n_gram = 2
        count = Counter(ngrams(text.split(), n_gram)).most_common()
        bg = dict(count).keys()
        for bigram in bg:
            if bigram in freq_lists_bg['pos']:
                counts['pos'] += 1
            if bigram in freq_lists_bg['neg']:
                counts['neg'] += 1
    return counts.most_common()

In [287]:
test_texts = random.choice(control_group)
print(test_texts)

Оформляли ипотеку на загородный дом в г. Красноярске. Хотелось бы выразить огромное спасибо сотрудникам банка- О-ву Илье и К-ву Кириллу, которые на протяжении всей сделки, начиная от заявки и до подписания договора, консультировали нас и помогли оформить ипотеку на очень выгодных условиях и по выгодной для нас программе, рассмотрев заявку индивидуально!! Побольше бы таких сотрудников, кто выполняет свою работу на 110% ))


In [297]:
detect_comment(freq_lists, test_texts, freq_lists_bg)

[('pos', 6), ('neg', 1)]

In [303]:
def test_detect_comment(freq_lists, test_size, freq_lists_bg):
    results = []  # результаты
    gold = []     # исходная эмоция
    for emotion in ['pos', 'neg']:
        for text in control_dict[emotion]:
            predicted_emotion = detect_comment(freq_lists, text, freq_lists_bg)[0][0]
            results.append(predicted_emotion)
            gold.append(emotion)
    print("RESULTS:")
    print("%d languages" % 2)
    print("Test size: %d texts per emotion" % test_size)
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [304]:
test_detect_comment(freq_lists, 10, freq_lists_bg)

RESULTS:
2 languages
Test size: 10 texts per emotion
Accuracy: 0.8600


Accuracy поднялась с 0.8163 (считалась в файле предыдущего дз) до 0.86. 